# 오차역전파법으로 구한 기울기 검증하기

(p. 184)


Chapter4~5를 통해 기울기<sup>gradient</sup>를 구한느 방법 두 가지를 공부함  
> (1) 수치 미분(p.121) - 계산 느림 

> (2) 해석학 미분 &rarr; backpropagation with computational graph
  * 오류역전파법을 사용하면 매개변수가 많아도 효율적으로 계산 



그래서 이제부터는 gradient decent를 위한 기울기 계산은 bacpropagation을 사용함 

# Q. 그러면 수치 미분은 아무 쓸모가 없는거임? 

모든 함수를 전부 해석학적으로 미분할 수 있는것도 아니고... <br/>
정말 무쓸모인가?  <br/>

<span style="color:red">No No No</span> <br/>
> 수치 미분은 bacpropagation을 정확히 구현했는지 확인하기 위해 필요함 


__i.e)__ network model에 대한 <span style="color:blue">backpropagation 구현을 잘 했는지 검사</span>할 때 씀 

# 기울기 검증(gradient check)

__수치 미분의 장점__ : 구현이 쉽다 
> * 수치 미분을 코딩할 때는 버그가 숨어 있기 어려움 <br/>

backpropagation은 계층 블록을 쌓다보면 복잡해져서 사람이 실수할 수 있음.<br/>
그래서 <span style="color:red">수치 미분의 결과</span>와 <span style="color:blue">backpropagation의 결과</span>를 비교하여 backpropagation을 제대로 구현했는지 검증함.
> 이같이 두 방식으로 구한 기울기가 일치함<sup>(__엄밀히 말해 거의 같음__)</sup>을 확인하는 작업을 __기울기 확인__<sup>gradient check</sup>이라고 함

### gradient check 코드 구현 

In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정

In [2]:
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

[TwoLayerNet](https://github.com/DoranLyong/Start_deepLearning_from_floor-/blob/master/Ch5_Backpropagation/L7-2_Neural_Network_with_Backpropagation.ipynb) 클래스 정의 

#### 데이터 불러오기 

In [3]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

[load_mnist](https://github.com/DoranLyong/Start_deepLearning_from_floor-/blob/master/Ch3_NeuralNet/L13_MNIST_data_img_load_into_numpy.ipynb) 함수 (p. 97)

#### 신경망 생성 

In [4]:
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)


# 계산 검증을 위해 일부 데이터만 불러옴 
x_batch = x_train[:3]    # 훈련 데이터 
t_batch = t_train[:3]    # 라벨링 데이터 


#### 기울기 구하기 

In [5]:
grad_numerical = network.numerical_gradient(x_batch, t_batch)   # 수치 미분 
grad_backprop = network.gradient(x_batch, t_batch)              # 해석 미분 by backpropagation 

각 방식으로 $\frac{\partial L}{\partial \mathbf{W}}$ 구함 

### 각 가중치 및 편향의 절대 오차의 평균
각 가중치 및 편향의 차이의 절대값을 구한 후, 그 절대값들의 평균을 계산 
> average( \| 역전파가중치 - 수치미분가중치 \| )

> average( \| 역전파편향 - 수치미분편향  \| )


In [6]:
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) ) # 
    print(key + "에 대한 차이"+ ":" + str(diff))

W1에 대한 차이:3.3996849984351894e-10
b1에 대한 차이:2.1597023282661324e-09
W2에 대한 차이:4.837028355711433e-09
b2에 대한 차이:1.3998314324914495e-07


__결과 해석__ (p. 185) <br/> 

예를 들어 1층의 가중치 매개변수 W1에 대해 비교했을 때, 오차는 0.0000000xxxx 로 매우 낮음 
> 이는 수치 미분과 오차역전파법으로 구한 기울기의 차이가 매우 작음을 의미 
  * backpropagation이 잘 구현됨 
  
### backpropagation이 잘 설계된것을 확인 했으니 이제 믿고 사용하면 됨 

In [7]:
grad_numerical.keys()

dict_keys(['W1', 'b1', 'W2', 'b2'])

In [8]:
print(grad_backprop['b1'])

[ 0.          0.          0.00252762 -0.00170557  0.         -0.00187136
  0.00043034 -0.00470567  0.0033343   0.          0.          0.00096062
 -0.00074439  0.          0.00574064  0.00044227  0.00413647  0.00084012
  0.00274844 -0.00169966 -0.0049455   0.00708124 -0.00189469 -0.00365544
 -0.00205302  0.         -0.01014807 -0.00467748  0.          0.00176041
 -0.00072848  0.0071463  -0.00216384  0.00147821  0.00114942 -0.00253224
  0.         -0.00735872  0.00203971 -0.00233744 -0.00150379  0.
  0.00083065  0.0018489  -0.00103975 -0.00293315  0.          0.00140423
  0.         -0.0033612 ]


In [9]:
print(grad_numerical['b1'])

[ 0.          0.          0.00252762 -0.00170557  0.         -0.00187135
  0.00043034 -0.00470567  0.0033343   0.          0.          0.00096062
 -0.00074439  0.          0.00574063  0.00044227  0.00413647  0.00084012
  0.00274844 -0.00169965 -0.0049455   0.00708123 -0.00189469 -0.00365544
 -0.00205302  0.         -0.01014806 -0.00467748  0.          0.00176041
 -0.00072848  0.00714629 -0.00216383  0.00147821  0.00114942 -0.00253223
  0.         -0.00735871  0.00203971 -0.00233744 -0.00150378  0.
  0.00083065  0.00184889 -0.00103975 -0.00293314  0.          0.00140423
  0.         -0.0033612 ]
